## Extraktionsbaserad textsammanfattare med olika rankningsmått 

#### Imports

In [59]:
# Imports --> TODO fixa ordning 
# Web scrapping -->  module för att ladda ner artiklar 
from newspaper import Article
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import nltk 
import ssl
import re  # används ej någonstans (tror jag) / björn
import regex as rex # 


# Summarization length of original text
percentage = 0.15
n_sentences = 3

# Fixes some errors, found online at https://github.com/gunthercox/ChatterBot/issues/930#issuecomment-322111087
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

#### Input

In [60]:
#text = "https://www.aftonbladet.se/nyheter/a/kE6ExL/sd-far-tunga-poster-i-utskotten"
#text = 'https://www.svt.se/nyheter/utrikes/stall-dina-fragor-om-kriget-till-svt-s-utrikesreportrar'
#text = "https://www.aftonbladet.se/nyheter/a/kE6j0a/uppgifter-viktigt-fynd-i-jakten-pa-mordarna-i-sodertalje"
text = 'https://www.svt.se/nyheter/utrikes/foi-expert-varnar-for-allvarliga-luckor-i-undervattensbevakningen'

In [61]:
article = Article(text, language='sv')
article.download()
article.parse()
text = article.text
homepage = article.meta_data['al']['ios']['app_name']
link = article.url
title = article.title

# Beroende på vilken hemsida nyheten kommer ifrån kan titeln och texten inehålla delar av sidan man egentligen inte bryr sig om
# T.ex. från aftonbladet är titeln med i texten och texten innehåller en mening som: "publicerad: 30 sep", man kan ta bort detta men 
# det blir om vi får tid över.

#print('Title:' , article.title, '\n\nText: \n', text)

# Preprocessing 

## Overview
### Calculate number of sentences to keep

### List 1 - sentences
* Varje mening separat

### Dataframe - scores
#### columns are the score of each ranking measure
* Baseline
* Headings
* TF/IDF-score
* NER
* ~~Class~~ //Om vi har tid för ML

### List 2 -> Cleaned for Stop Words 
* Varje mening separat 

###

##### Original Sentences List

In [62]:
# removes endlines:
from token import NEWLINE

org_sentences = text.replace('\n\n', '. ')
# creates some exceptions from above rule
org_sentences = org_sentences.replace('.. ', '. ')
org_sentences = org_sentences.replace(':. ', ': ')
org_sentences = org_sentences.split('. ')

sentences_to_remove = []
for i, sentence in enumerate(org_sentences): 
    if sentence == title: 
        sentences_to_remove.append(sentence)
    if 'publicerad:' in sentence.lower(): 
        sentences_to_remove.append(sentence)

org_sentences = [sentence for sentence in org_sentences if sentence not in sentences_to_remove]

org_sentences[0:5]

['Säkerhetspolisen konstaterar att det har skett detonationer vid Nord Stream 1 och 2 i svensk ekonomisk zon',
 'Det nämns dock inget om antalet sprängningar',
 '– Att genomföra det här sabotaget pekar på sårbarheten i den här typen av anläggningar och det är ett sätt att utöva påtryckningar i sig: ”Det här är vad vi kan göra mot er!”, säger Niklas Granholm, forskningsledare vid Totalförsvarets forskningsinsitut (FOI)',
 'En av fyra ledningar intakt',
 'Danska statliga Energistyrelsen uppgav dock på onsdagen att en av Nord Streams fyra rörledningar, Nord Stream 2 Linje B, inte är skadad – trots de kraftiga explosionerna']

##### Dataframe

In [63]:
index = range(org_sentences.__len__())
columns = ['Baseline', 'Headings', 'TF', 'NER']
scores = pd.DataFrame(index=index, columns=columns)
scores.fillna(0, inplace=True)
scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   Baseline  24 non-null     int64
 1   Headings  24 non-null     int64
 2   TF        24 non-null     int64
 3   NER       24 non-null     int64
dtypes: int64(4)
memory usage: 896.0 bytes


#### Create spacy doc object

In [64]:
import spacy
# Credit to Explosion for sv_core_news_sm --> https://github.com/explosion 
# "lemmatization accuracy 0.95"
# Create spacy nlp object 
nlp = spacy.load("sv_core_news_sm") # nlp used by lemmatizer()

#### Lemmatizer

In [65]:
# den --> det 
# noterar detta att det är en dålig lemmatiserare --> språkbanken stanza / lemmy / kth...  

def lemmatizer(list_of_strings):
    lemmatized_sentences = []
    lemmatized_sentence = ''
    for i in range(len(list_of_strings)): 
        sentence_to_lemmatize = nlp(list_of_strings[i])
        for token in sentence_to_lemmatize:
            lemma = token.lemma_
            lemmatized_sentence += lemma + ' '  
        
        lemmatized_sentences.append(lemmatized_sentence)
        lemmatized_sentence = '' 

    return lemmatized_sentences

# Created Lemmatized DS
lemmatized_org_sentences = lemmatizer(org_sentences)
print(lemmatized_org_sentences)

['säkerhetspolis konstatera att den ha ske detonation vid Nord stream 1 och 2 i svensk ekonomisk zon ', 'en nämn dock ingen om antal sprängning ', '– att genomföra en här sabotag peka på sårbarhet i en här typ av anläggning och den vara en sätt att utöva påtryckning i sig : ” den här vara vad vi kunna göra mot ni ! ” , säga Nikla Granholm , forskningsledare vid Totalförsvarets forskningsinsitut ( FOI ) ', 'man av fyra ledning intakt ', 'danska statlig Energistyrelsen uppge dock på onsdag att en av Nord Streams fyra rörledning , Nord stream 2 Linje B , inte vara skada – trots en kraftig explosionerna ', '– om en av en fyra gasrör vara intakt , så ge den Ryssland möjlighet att utöva påtryckning mot importör – den vilja säga Västeuropa , säga Nikla Granholm ', 'danska Energistyrelsens twitterkonto foto : Twitter 5 oktober 2022 ', '” påtryckning att öppna ” ', 'en övrig tre rör : Nord stream 1 Linje A och Linje B samt Nord stream 2 Linje A uppge vara obrukbara efter detonationere ', 'Uppgi

#### Proper Nouns

In [66]:
# För bättre täckning på NER 
def proper_nouns(list_of_strings):
    proper_nouns = set()
    for i in range(len(list_of_strings)): 
        sentence_to_pos = nlp(list_of_strings[i])
        for token in sentence_to_pos: 
            token_str = token.text
            if token.pos_ == "PROPN" and len(token_str) > 1:
                proper_nouns.add(token_str.strip())
    return proper_nouns

print(proper_nouns(org_sentences))


{'Sverige', 'Energistyrelsen', 'Niklas', 'SOSUS', 'Energistyrelsens', 'Östersjön', 'Grafik', 'Danmarks', 'Nord', 'Foto', 'SVT', 'Västeuropa', 'Streams', 'Tyskland', 'FOI', 'Granholm'}


#### Named Entities

In [67]:
def named_entity_recognition(list_of_strings):
    doc = nlp(' '.join(list_of_strings))
    # Convert tuple[Span] to str
    named_entities = doc.ents.__str__()
    # Remove string parenthesis 
    named_entities = named_entities[1:len(named_entities) - 1]
    # Create list of strings
    named_entities = named_entities.split(',')
    
    named_entities_set = set()
    for entity in named_entities: 
        named_entities_set.add(entity.strip())
    return named_entities_set

### Stop Word Filtering

In [68]:
# Inspired by https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

def get_swe_stop_words(): 
    swe_stop_words = set(stopwords.words('swedish'))
    # Not stop word cleaning 
    swe_stop_words.update([',', '"', ':', '-', '–', '”'])
    return swe_stop_words

def stop_word_filtering(list_of_strings):
    word_tokens = word_tokenize(' '.join(list_of_strings))
    filtered_sentence_into_singletons = [w for w in word_tokens if not w.lower() in get_swe_stop_words()]
    return filtered_sentence_into_singletons

#### Frequency Distribution 

In [69]:
def frequency_distribution(list): 
    fdist = FreqDist(word.lower() for word in word_tokenize(' '.join(list)))
    return fdist

In [70]:
filtered_words = stop_word_filtering(lemmatized_org_sentences) 
fdist = frequency_distribution(filtered_words)

# Ranking Measures

### Baseline
(1/N --> n=ordning mening kommer i dvs. första meningen får N=1 -> 1/1, andra meningen får N=2 -> 1/2, osv.)

In [71]:
# Ranking metric 1 --> Baseline
for i, score in enumerate(scores['Baseline']) :
    scores['Baseline'][i] = 1/((i+1))
scores.describe()

,Baseline,Headings,TF,NER
count,24.000000,24.0,24.0,24.0
mean,0.157332,0.0,0.0,0.0
std,0.209559,0.0,0.0,0.0
min,0.041667,0.0,0.0,0.0
25%,0.054825,0.0,0.0,0.0
50%,0.080128,0.0,0.0,0.0
75%,0.148810,0.0,0.0,0.0
max,1.000000,0.0,0.0,0.0


### Headings

In [72]:
# Ranking metric 2 --> Headings
# Sets all 'Headings' scores to 0, mostly for testing so i can run this multiple times, 
# but also to make sure nothing weird has happened earlier in the code.
scores['Headings'] = 0
for i, sentence in enumerate(org_sentences): # Lemmatizerad? 
    for word in article.title.split(' '):
        if word in sentence:
            scores.at[i, 'Headings'] += 1
scores.describe()

,Baseline,Headings,TF,NER
count,24.000000,24.000000,24.0,24.0
mean,0.157332,1.166667,0.0,0.0
std,0.209559,0.481543,0.0,0.0
min,0.041667,0.000000,0.0,0.0
25%,0.054825,1.000000,0.0,0.0
50%,0.080128,1.000000,0.0,0.0
75%,0.148810,1.000000,0.0,0.0
max,1.000000,2.000000,0.0,0.0


Sverigedemokraterna få ordförandepost i riksdag justitie och utrikesutskott 
[' få ', 'ordförandepost ', 'riksdag ', 'utrikesutskott ']

* bara ord i lemma 
* ord i lemma med efter letters
* ord i lemma med före letters --> x
* före + lemma + efter 

In [73]:
preprocessed_title = stop_word_filtering(lemmatizer(title.split(' ')))
p = rex.compile(r"\b\S*\L<words>\S*\b | \b\L<words>\b | \b\L<words>\S*\b", re.IGNORECASE, words=preprocessed_title)
p.findall

for i, sentence in enumerate(lemmatized_org_sentences):    
    #print(sentence)
    matches = p.findall(sentence)
    #print(matches)
    scores.at[i, "Headings"] = len(matches)

scores.describe()

,Baseline,Headings,TF,NER
count,24.000000,24.000000,24.0,24.0
mean,0.157332,0.041667,0.0,0.0
std,0.209559,0.204124,0.0,0.0
min,0.041667,0.000000,0.0,0.0
25%,0.054825,0.000000,0.0,0.0
50%,0.080128,0.000000,0.0,0.0
75%,0.148810,0.000000,0.0,0.0
max,1.000000,1.000000,0.0,0.0


## TERM FREQUENCY 


In [74]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
#Term Frequency
def termfreq(sentence, word):
    N = len(sentence)
    occurance = len([token for token in sentence if token == word])
    return occurance/N

In [75]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
#Inverse Document Frequency
def inverse_doc_freq(word):
    try:
        word_occurance = fdist.get(word) + 1
    except:
        word_occurance = 1
    return np.log(len(lemmatized_org_sentences)/word_occurance)

In [76]:
# Borrowed https://www.askpython.com/python/examples/tf-idf-model-from-scratch (with adjustments)
# TF*IDF-score 
def tf_idf(sentence):
    score = 0
    #tf_idf_vec = np.zeros((len(filtered_words),))
    for word in sentence:
        tf = termfreq(sentence,word)
        idf = inverse_doc_freq(word)
         
        score += tf*idf
        #tf_idf_vec[index_dict[word]] = value 
    return score

for i, sentence in enumerate(lemmatized_org_sentences):
    scores.at[i, 'TF'] = tf_idf(sentence)

#scores.describe()
#scores['TF'].idxmax()
#scores.loc[scores['TF'] == scores['TF'].median()]

### TF*IDF-score

https://forketyfork.medium.com/latex-math-formulas-a-cheat-sheet-21e5eca70aae

In [77]:
# Ranking metric 3 --> TF*IDF

# Term Weights --> Calculate importance of single words in text/doc
# Binary term weights --> document specific
# TF*IDF term weights --> document-collection specific 

# Assign weights to each dimension (attr/word) of each sentence (record/example) 

# Term Frequency (TF-score) --> TFij == frequency of the jth term in in the ith doc 

# Inverse Document Frequency 
# idf-score of the jth term measures the uniqueness of the jth term in the collection of documents
# IDFj = log(M / Nj)
#
# M = total num of docs in collection 
# Nj is the number of documents that contain the jth term

# HIGH TF*IDF-score 
# Word frequent in document && Occur in few documents of the collection 
# LOW TF*IDF-score
# Not present in document || present in all documents of the collection 

### NER 

In [78]:
scores['NER'] = 0
named_entities = named_entity_recognition(org_sentences)
proper_nouns = proper_nouns(org_sentences)
ner_unique = named_entities.union(proper_nouns)
print(ner_unique)


{'Niklas Granholm', 'Sverige', 'Energistyrelsen', 'Östersjön', 'Danska statliga Energistyrelsen', 'SOSUS', 'Energistyrelsens', 'Nord Streams', 'Grafik', 'Nord', 'Västeuropa', 'Grafik SVT Sensorer', 'Tyskland', 'Danska Energistyrelsens', 'Granholm', 'Niklas', 'Stream 2', 'Totalförsvarets', 'Linje B samt Nord', 'Danmarks', 'Foto', 'Danmarks Energistyrelse', 'SVT', 'Streams', 'Ryssland', 'FOI'}


In [79]:
p = rex.compile(r"\L<words>", words=ner_unique)

for i, sentence in enumerate(org_sentences):
    matches = p.findall(sentence)
    scores.at[i, "NER"] = len(matches)

scores.describe()
scores.head()


,Baseline,Headings,TF,NER
0,1.000000,0,25.215988,1
1,0.500000,0,13.821866,0
2,0.333333,0,57.927523,3
3,0.250000,0,8.176898,0
4,0.200000,0,37.478216,4


# Combination Function

#### Standardize

In [80]:
# Standardize
scores_standardized = StandardScaler.fit_transform(self=StandardScaler(), X=scores)
scores_standardized = pd.DataFrame(scores_standardized, columns=columns)
scores_standardized['Baseline'] *= 1.5
scores_standardized['Headings'] *= 2.5
scores_standardized['TF'] *= 0.3
scores_standardized


,Baseline,Headings,TF,NER
0,6.161452,-0.521286,0.046203,-0.346410
1,2.505535,-0.521286,-0.200388,-1.039230
2,1.286896,-0.521286,0.754145,1.039230
3,0.677576,-0.521286,-0.322557,-1.039230
4,0.311984,-0.521286,0.311582,1.732051
5,0.068257,-0.521286,0.216061,1.039230
6,-0.105835,-0.521286,-0.161226,0.346410
7,-0.236403,-0.521286,-0.323294,-1.039230
8,-0.337956,-0.521286,0.173746,1.039230
9,-0.419199,-0.521286,0.211173,0.346410


#### Calculate Summarization Length (number of sentences)

In [81]:
num_of_org_sentences = len(org_sentences)
summarization_num_sentences = round(num_of_org_sentences * percentage)

print("summarization: ", summarization_num_sentences, "\noriginal: ", num_of_org_sentences)

summarization:  4 
original:  24


#### Combine
* Combine the scores into one overall score
* add weight and/or ML if time allows

In [82]:
# Combination Function
# Här ligger ML om vi gör det  

#final_score = scores_standardized.drop('TF', axis=1).sum(axis=1)
final_score = scores_standardized.sum(axis=1)
best_sentences = final_score.nlargest(summarization_num_sentences, keep='all').index.values
print(best_sentences)

print(org_sentences[0])


[21  0  2  4]
Säkerhetspolisen konstaterar att det har skett detonationer vid Nord Stream 1 och 2 i svensk ekonomisk zon


In [83]:
# Similarity heurstic 
# Rubrik --> Hur mycket lika --> Ta bort redundans 
# 5 fem väldigt lika --> heuristik välja bort något av det     

# Assemble output 
* Reassemble according to overall score ranking
* Output summarization 

In [84]:
# Percentage length of original text --> summarization 
print("Percentage:\n")
print(title, "\n")
for i in best_sentences: 
    print(org_sentences[i])

print("\n\n")

# N sentences 
print("N sentences:\n")
for i in best_sentences[0:n_sentences]: 
    print(org_sentences[i])

Percentage:

FOI-expert varnar för bristande spaning i Östersjön 

Under kalla kriget lades linjer med undervattensmikrofoner (SOSUS) ut i delar av Östersjön, vilket SVT rapporterat
Säkerhetspolisen konstaterar att det har skett detonationer vid Nord Stream 1 och 2 i svensk ekonomisk zon
– Att genomföra det här sabotaget pekar på sårbarheten i den här typen av anläggningar och det är ett sätt att utöva påtryckningar i sig: ”Det här är vad vi kan göra mot er!”, säger Niklas Granholm, forskningsledare vid Totalförsvarets forskningsinsitut (FOI)
Danska statliga Energistyrelsen uppgav dock på onsdagen att en av Nord Streams fyra rörledningar, Nord Stream 2 Linje B, inte är skadad – trots de kraftiga explosionerna



N sentences:

Under kalla kriget lades linjer med undervattensmikrofoner (SOSUS) ut i delar av Östersjön, vilket SVT rapporterat
Säkerhetspolisen konstaterar att det har skett detonationer vid Nord Stream 1 och 2 i svensk ekonomisk zon
– Att genomföra det här sabotaget pekar på

In [85]:
# Newspaper Summarization
article.nlp()
print("\nNewspaper3k: \n", article.summary)


Newspaper3k: 
 Säkerhetspolisen konstaterar att det har skett detonationer vid Nord Stream 1 och 2 i svensk ekonomisk zon.
En av fyra ledningar intaktDanska statliga Energistyrelsen uppgav dock på onsdagen att en av Nord Streams fyra rörledningar, Nord Stream 2 Linje B, inte är skadad – trots de kraftiga explosionerna.
Danska Energistyrelsens twitterkonto Foto: Twitter 5 oktober 2022”Påtryckning att öppna”De övriga tre rören: Nord Stream 1 Linje A och Linje B samt Nord Stream 2 Linje A uppges vara obrukbara efter detonationerna.
Uppgifterna innebär att Nord Stream 2 kanske kan öppnas, trots att den på grund av amerikanska sanktioner årsskiftet 2019/2020 aldrig blev godkänd av tyska myndigheter.
– Vi hade Nord Stream 1 i funktion, medan Nord Stream 2 aldrig togs i bruk.


In [86]:
summarization = link + '\n\n' + 'Title: ' + '\n' + title + '\n\n' + 'Summarization:' + '\n'
for sentence in best_sentences: 
    summarization += org_sentences[sentence] + '\n'
summarization += '\n\n' + 'Newspaper3k:' + '\n' + 'Title: ' + '\n' + title + "\n\n" + article.summary

In [87]:
import os 
FILEPATH = f'./summarizations/{homepage}/'
filename = f"{FILEPATH}{title}.txt"
os.makedirs(os.path.dirname(filename), exist_ok=True)

with open(filename, 'w') as f:
     f.write(summarization)